# HDFS

HDFS is het distributed file system van Hadoop dat de basis vormt voor een breed gamma van applicaties, waaronder MapReduce.
Dit framework maakt het mogelijk om niet-gespecialiseerde hardware te gebruiken om eenvoudig datacenters op te zetten of rekenclusters te beheren.
HDFS bereikt dit doel op een aantal manieren:
* Ten eerste wordt een file opgedeeld in verschillende blokken. Deze worden verdeeld over verschillende computers zodat code meerdere delen van het bestand kan gebruiken in parallel om zo de benodigde rekenkracht te verdelen over meerdere toestellen.
* Daarnaast worden de blokken ook gedupliceerd om zo fout-toleranter te zijn in het geval van een crash (hardware of software), stroomstoring, netwerkonderbreking.

Om dit te bereiken gebruikt Hadoop een Master-Slave architectuur dat bestaat uit een enkele namenode (master) en meerdere datanodes (slaves).
De namenode houdt bij hoeveel datanodes er actief zijn, welke blokken ze hebben en welke blokken bij welke file horen.
Indien er een datanode crasht gaat deze server dit detecteren en dit oplossen door de nodige blokken te kopieren van een andere datanode zodat er steeds voldoende kopies in het systeem aanwezig zijn.
Bij elke actie die uitgevoerd moet worden in het HDFS moet er steeds gevraagd worden aan de namenode welke blokken op welke datanodes we nodig hebben voor de gewenste file uit te lezen of code voor uit te voeren.
Het is dus duidelijk dat deze namenode een single-point-of-failure is wat ideaal is voor de availability van de cluster.
Dit kan opgelost worden door HDFS te runnen in een high-availability mode wat ervoor zorgt dat er steeds een backup aanwezig is voor de namenode die zeer snel de werking kan overnemen.
Deze structuur maakt het eenvoudig om aan horizontal scaling te doen door extra servers toe te voegen zonder dat er downtime is voor de hele cluster.

Dit resulteer in de volgende kenmerken van HDFS:
* High Throughput
* Scalability
* High Availability
* Data Reliability
* Fault Tolerance

## Starten en stoppen van de cluster

De cluster kan gestart worden door de volgende commando's uit te voeren in de commandline:
* start-dfs.sh
* start-yarn.sh

Eenmaal dat de cluster gestart is kunnen de lopende java applicaties gecontroleerd worden door middel van het commando jps.
Dit kan ook uitgevoerd worden vanuit deze jupyter notebook door onderstaande code-cell uit te voeren.

In [2]:
# uitroepingsteken zegt tegen jupyter notebook dat dit op de terminal uitgevoerd wordt

!start-dfs.sh
!start-yarn.sh
#!start-all.sh
!jps

Starting namenodes on [localhost]
Starting datanodes
Starting secondary namenodes [bigdata-VirtualBox]
Starting resourcemanager
Starting nodemanagers
22529 DataNode
22403 NameNode
22707 SecondaryNameNode
23237 Jps
22953 ResourceManager
23087 NodeManager


Ook heeft de namenode een website waar naartoe kan gegaan worden om meer informatie over de cluster te bekijken. De standaardurl is [http://localhost:9870](http://localhost:9870).

Om de cluster terug te stoppen moeten in de terminal of de notebook de volgende commando's uitgevoerd worden.
* stop-dfs.sh
* stop-yarn.sh


In [ ]:
#!stop-yarn.sh
#!stop-dfs.sh

## Communiceren met het HDFS

Er zijn verschillende manieren om dit distributed bestandssysteem te gebruiken.
Ten eerste kan je gebruik maken van een command line interface (CLI) om bestanden uit te lezen, op te laden, ...
Daarnaast zijn er ook wrappers voor verschillende talen die toelaten om rechtstreeks vanuit code te interageren met het HDFS.
De voordelen van deze wrappers over een CLI zijn:
* Flexibiler om te interageren in applicaties.
* Gebruiksvriendelijker en gemakkelijker om te automatiseren.
* Eenvoudiger in onderhoud en te debuggen
* Volledige functionaliteit van een programmeertaal kan gebruikt worden zoals OO.

### Instantiering van een client

(Niet nodig als je de commandline gebruikt)

Veel verschillende talen beschikken over een wrapper om te communiceren met een HDFS.
In deze cursus gaan we gebruik maken van [de pydoop](https://crs4.github.io/pydoop/index.html) wrapper.
De eerste stap in het gebruik van de wrapper is te bepalen hoe de namenode van het hdfs gevonden kan worden.
Dit gebeurt als volgt:

In [ ]:
!pip install pydoop

In [1]:
import pydoop.hdfs as hdfs

In [3]:
# connect met het locale filesysteem
localFs = hdfs.hdfs(host='')
# connect met het hdfs (default port is 9000)
client = hdfs.hdfs(host="localhost", port=9000)

client.capacity()


2022-02-17 09:55:38,051 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


31040933888

### Aanmaken van files en directories

Om nu bestanden en folders aan te maken op dit distributed file systeem kunnen onderstaande functies gebruikt worden.

In [9]:
# er wordt steeds vertrokken van de folder /user/bigdata

#client.exists("09_Streaming")

# aanmaken via commandline
# !hdfs dfs -mkdir -p 03_HDFS_term

path = "03_HDFS"
if not client.exists(path):
    client.create_directory(path)
    
# zo moet je niet altijd het pad naar de directory meegeven
# proefondervindelijk kan het zijn dat niet alle commando's hiernaar luisten
client.set_working_directory(path)

In [40]:
# uploaden van files
#hadoop fs -put /path/in/linux /path/in/hdfs/with/filename
#hadoop fs -put davinci_notebooks.txt 03_HDFS_term/notebooks.txt
#localFs.copy("ulysses.txt", client, "03_HDFS/book.txt")
# bij move is het origneel weg
localFs.move("book.txt", client, "03_HDFS/book2.txt")

In [13]:
# downloaden van files
#!hadoop fs -get 03_HDFS_term/notebooks.txt
client.copy("03_HDFS/book.txt", localFs, "book.txt")

0

### Bekijken van het filesysteem

In [33]:
#client.list_directory(".")
# niveau hoger
#client.list_directory("..")
# data used
#client.used()
client.set_working_directory("/user/bigdata/03_HDFS")
client.working_directory()
client.list_directory(".")
client.default_block_size()

# de eerst 5 regels
f = client.open_file("book.txt")
for i in range(5):
    print(f.readline())
f.close()

b'\xef\xbb\xbfThe Project Gutenberg eBook of Ulysses, by James Joyce\n'
b'\n'
b'This eBook is for the use of anyone anywhere in the United States and\n'
b'most other parts of the world at no cost and with almost no restrictions\n'
b'whatsoever. You may copy it, give it away or re-use it under the terms\n'


True

### Aanpassen van het filesysteem

In [35]:
# rename
#client.rename("book.txt", "boek.txt")
!hadoop fs -mv 03_HDFS/boek.txt 03_HDFS/book.txt

In [37]:
# delete
#client.delete("book2.txt")
!hdfs dfs -rm 03_HDFS/book.txt

Deleted 03_HDFS/book.txt


In [39]:
#client.delete("/user/bigdata/03_HDF_term")
client.delete("/user/bigdata/03_HDFS_term")

### Oefening

Schrijf python code dat controleert of een directory bestaat in het hdfs.
Indien nee wordt de directory aangemaakt.
Indien ja, worden alle files in de directory verwijderd om van een lege directory te starten.
Upload daarna een tekst-bestand naar keuze.

In [46]:
client.set_working_directory("/user/bigdata")
path = "03_HDFS"
if not client.exists(path):
    client.create_directory(path)
else:
    # dit gaat de volledige directory verwijderen
    #client.delete(path + "/")
    
    # dit is geen geldig pad dus error
    # client.delete(path + "/*")
    
    for f in client.list_directory("."):
        # do not delete input file
        if(f["name"] != "dataset.csv"):
            client.delete(f["name"])

localFs.copy("ulysses.txt", client, "03_HDFS/book.txt")

0